aaprint(9)

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
 
# Start Spark session
spark = SparkSession.builder.appName("ShowSchedule").getOrCreate()
 
# Define the schema
schema = StructType([
    StructField("show_date", StringType(), True),
    StructField("show_status", StringType(), True)
])
 
# Define the data
data = [
    ("01/06/2020", "Booked"),
    ("02/06/2020", "Booked"),
    ("03/06/2020", "Booked"),
    ("04/06/2020", "Available"),
    ("05/06/2020", "Available"),
    ("06/06/2020", "Available"),
    ("07/06/2020", "Booked")
]
 
# Create DataFrame
df = spark.createDataFrame(data, schema)

print(df.printSchema())
# Show the DataFrame
df.show()

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 27, Finished, Available, Finished)

root
 |-- show_date: string (nullable = true)
 |-- show_status: string (nullable = true)

None
+----------+-----------+
| show_date|show_status|
+----------+-----------+
|01/06/2020|     Booked|
|02/06/2020|     Booked|
|03/06/2020|     Booked|
|04/06/2020|  Available|
|05/06/2020|  Available|
|06/06/2020|  Available|
|07/06/2020|     Booked|
+----------+-----------+



In [24]:
display(df.orderBy(f.col("show_date").asc()))

StatementMeta(, 7a0cda67-c1d2-46bb-9ebb-ab08f2ffaeb5, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 788f8ee3-f94b-41e6-baba-41f7318fb7ff)

In [26]:
df1 = df.withColumn("lag",f.col("show_status"))
display(df1)

StatementMeta(, 7a0cda67-c1d2-46bb-9ebb-ab08f2ffaeb5, 29, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 072dc09a-9257-4e35-9758-8b3daa50a8c3)

In [43]:
df1 = df.withColumn("lag",f.lag(f.col("show_status")).over(Window.orderBy(f.col("show_date"))))
display(df1)

StatementMeta(, 7a0cda67-c1d2-46bb-9ebb-ab08f2ffaeb5, 46, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2adfe8b3-8320-4280-b3b2-93f57c4cd293)

In [59]:
df1 = df.withColumn("change",f.when(f.col("show_status") != f.lag(f.col("show_status")).over(Window.orderBy(f.col("show_date"))),1).otherwise(0))
display(df1)

StatementMeta(, 7a0cda67-c1d2-46bb-9ebb-ab08f2ffaeb5, 62, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4cdcf96b-fb03-4be2-ab0a-9cdf6acd8b11)

In [ ]:
df2= df1.withColumn("sum",f.sum("change").over(Window.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)))

df2.show()

In [72]:
df2= df1.withColumn("sum",f.sum("change").over(Window.rowsBetween(Window.unboundedPreceding, Window.currentRow)))
()
df2.show()

StatementMeta(, 7a0cda67-c1d2-46bb-9ebb-ab08f2ffaeb5, 75, Finished, Available, Finished)

+----------+-----------+------+---+
| show_date|show_status|change|sum|
+----------+-----------+------+---+
|01/06/2020|     Booked|     0|  0|
|02/06/2020|     Booked|     0|  0|
|03/06/2020|     Booked|     0|  0|
|04/06/2020|  Available|     1|  1|
|05/06/2020|  Available|     0|  1|
|06/06/2020|  Available|     0|  1|
|07/06/2020|     Booked|     1|  2|
+----------+-----------+------+---+



In [94]:
df3= df2.groupBy(f.col("sum"), f.col("show_status"))\
        .agg(f.min(f.col("show_date")).alias("min")\
            ,f.max(f.col("show_date")).alias("max")
            )
display(df3)

StatementMeta(, 7a0cda67-c1d2-46bb-9ebb-ab08f2ffaeb5, 97, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6e1342b8-ad8e-4db3-817c-cdbc1c15dae0)

In [95]:
display(df3.drop(f.col("sum")))

StatementMeta(, 7a0cda67-c1d2-46bb-9ebb-ab08f2ffaeb5, 98, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d531ddfc-3ae2-440b-bdcc-968a4e38e57a)

In [66]:
df2= df1.withColumn("sum",f.count("change").over(Window.partitionBy(f.col("show_status"))))

df2.show()

StatementMeta(, 7a0cda67-c1d2-46bb-9ebb-ab08f2ffaeb5, 69, Finished, Available, Finished)

+----------+-----------+------+---+
| show_date|show_status|change|sum|
+----------+-----------+------+---+
|04/06/2020|  Available|     1|  3|
|05/06/2020|  Available|     0|  3|
|06/06/2020|  Available|     0|  3|
|01/06/2020|     Booked|     0|  4|
|02/06/2020|     Booked|     0|  4|
|03/06/2020|     Booked|     0|  4|
|07/06/2020|     Booked|     1|  4|
+----------+-----------+------+---+



In [3]:
from pyspark.sql import functions as f
from pyspark.sql.functions import window
from pyspark.sql.window import Window

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 5, Finished, Available, Finished)

In [14]:
display(df)

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 995efc9f-b17f-4ee9-b12a-d1b2c506f2f5)

In [13]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 15, Finished, Available, Finished)

In [16]:
df=df.withColumn("show_date",f.to_date(f.col("show_date"), "dd/MM/yyyy"))
display(df)

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4e4d27e9-dd4c-458e-973c-5b4661fadcfa)

In [17]:
dff=df.withColumn("rn",f.row_number().over(Window.partitionBy(f.col("show_status"))\
                                                 .orderBy(f.col("show_date"))\
                                           )
                  )
display(dff)

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 92cc783f-4506-4b09-a1a1-fe35d0c52e07)

In [19]:
dff2=dff.withColumn("date-rn",f.date_sub(f.col("show_date"),f.col("rn")))
display(dff2)

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 92dbcfb4-b1f8-4468-9fb8-71f77911118e)

In [21]:
dff3=dff2.groupBy(f.col("show_status"),f.col("date-rn"))\
         .agg(f.min(f.col("show_date")).alias("min"),
              f.max(f.col("show_date")).alias("max")
             )
display(dff3)

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 233f2f62-d9d7-4b8c-b0c6-009d4c7d8166)

In [24]:
dff3=dff3.drop("date-rn").orderBy(f.col("min"))
display(dff3)

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, df83cf0d-b0af-4f7a-8220-47c57287eab1)

In [29]:
ff=df.withColumn("rn",f.row_number().over(Window.orderBy(f.col("show_date"))))
ff2=ff.withColumn("rn2",f.row_number().over(Window.orderBy(f.col("show_date"))\
                                       .partitionBy(f.col("show_status"))))
display(ff2)

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 215cbcff-2ac8-4492-9a4f-03c8d205b0c6)

In [31]:
ff3=ff2.withColumn("diff",f.col("rn")-f.col("rn2"))
display(ff3)

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 33, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c46d97a6-3e93-4d3a-ac08-8dd795c21917)

In [35]:
ff4=ff3.groupBy(f.col("show_status"), f.col("diff"))\
       .agg(f.min(f.col("show_date")).alias("min"),
            f.max(f.col("show_date")).alias("max")
            )
ff4=ff4.drop("diff").orderBy("min")
display(ff4)

StatementMeta(, 6a1bb1c8-8592-4d9e-89cb-4ad902b78f32, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e38a5773-21b1-47dc-aa9b-75e43d24e5da)